<div style="direction:rtl;line-height:300%;">
<font face="XB Zar" size=5>
<div align=center>
<font face="B Titr" size=5>
<p></p><p></p>
بسمه تعالی
<p></p>
</font>
<p></p>
<font>
<br>
درس بازیابی پیشرفته اطلاعات
<br>
مدرس: دکتر بیگی
</font>
<p></p>
<br>
<font>
<b>تمرین دوم</b>
</font>
<br>
<br>
موعد تحویل: ... آبان <br>
<br>
<br>
<br>
<font>
دانشگاه صنعتی شریف
<br>
دانشکده مهندسی کامپیوتر
<br>
<br>
</font>
</div>
</font>
</div>

<div dir='rtl'>
<h1> 
مقدمه
</h1>
<p>
در این تمرین قصد داریم به مباحث زیر بپردازیم:
    <li>مدل‌های برداری</li>
    <li>امتیازدهی و ارزیابی سیستم بازیابی</li>
    <li>مدل‌های احتمالاتی</li>

دیتاست مورد استفاده در این تمرین را می‌توانید در کنار این فایل مشاهده کنید. همچنین لطفا پس از اتمام تمرین یک بار از اول تا آخر نوت‌بوک را اجرا کنید تا مطمئن باشید تمام سل‌ها به درستی کار می‌کنند. 

کتابخانه‌های مورد نظرتان را هم می‌توانید در اولین سل نوت‌بوک فراخوانی کنید. 
</p>
</div>

In [1]:
import math
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
from collections import defaultdict, deque, Counter
from tqdm.notebook import tqdm
from functools import lru_cache
import json

<div dir='rtl'>
<h1>1.
آماده‌سازی دیتاست
</h1>
<p>
با استفاده از تمرین قبل و عملیات‌هایی که در آنجا برای پیش‌پردازش متون پیاده‌سازی کردید، دیتاست داده شده را لود کنید. قرار است در ادامه با این دیتاست بخش‌های بعدی تمرین را پیاده‌سازی کنید. همچنین در صورتی که اجرای سل‌های بعدی برایتان طول کشید، می‌توانید آن را کوچک کنید. مثلا از ۷۰درصد دیتای آن استفاده کنید. 
</p>
</div>

In [2]:
# load the dataset
df = pd.read_csv('data.csv', index_col='paperId')
df = df.loc[:, ['title', 'abstract']]
df.fillna('', inplace=True)
df = df.reset_index().drop_duplicates(subset='paperId', keep='first').set_index('paperId')
df.head()

,title,abstract
paperId,,
40ea606185b59cd07b456cb1022d64bf41f5538d,Analysis on ground surface in ultrasonic face ...,
597c2d96c45e8ad83fc08e5d464d266b68f873ed,Measuring SARS-CoV-2 neutralizing antibody act...,The emergence of SARS-CoV-2 has created a need...
afbf330f0180320deff12fe42ded4f087b4a1811,A variational model for disocclusion,In this paper we study a variational approach ...
fa57e1cbaa46211499749cc75172e9ced26ad539,Helicoidally Arranged Polyacrylonitrile Fiber-...,"In this study, we demonstrate the use of paral..."
8780ce7c0d09d36d293def6be23e64b12c644169,The Interaction of SIRT4 and Calreticulin duri...,


In [3]:
# preprocess
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re
from nltk.tokenize import word_tokenize


class Preprocessor:
    def __init__(self):
        # Create a variable of stop words.
        self.stopwords = list(stopwords.words("english"))

    def preprocess(self, text):
        # The main function of the class.
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.normalize(words)
        words = self.remove_stopwords(words)
        words = self.remove_digits_and_chars(words)
        return words

    def normalize(self, words):
        # Normalize text (lower case, stemming, lemmatization, etc.)
        lemmatizer = WordNetLemmatizer()
        stemmer = PorterStemmer()

        words = [word.lower() for word in words]
        words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words]
        return words

    def remove_links(self, text):
        # Remove links
        return re.sub(r"https?\S+", "", text)

    def remove_punctuations(self, text):
        # Remove punctutaions
        return text.translate(
            str.maketrans(string.punctuation, " " * len(string.punctuation))
        )

    def word_tokenize(self, text):
        # Tokenize text
        words = word_tokenize(text)
        return words

    def remove_stopwords(self, words):
        # Remove stopwords
        words = [word for word in words if word not in self.stopwords]
        return words

    def remove_digits_and_chars(sef, words):
        words = [word for word in words if not word.isdigit() and word.isalpha()]
        words = [word for word in words if len(word) > 1]
        return words


preprocessor = Preprocessor()

corpus = defaultdict(lambda: {'title': [], 'abstract': []})
for document_index, preprocessed_row in tqdm(df.iterrows(), total=df.shape[0]):
    if document_index in corpus:
        print(document_index)
    corpus[document_index]['title'] = preprocessor.preprocess(preprocessed_row['title'])
    corpus[document_index]['abstract'] = preprocessor.preprocess(preprocessed_row['abstract'])
corpus[df.index[0]]

[nltk_data] Downloading package punkt to /home/ali/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ali/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


  0%|          | 0/6183 [00:00<?, ?it/s]

{'title': ['analysi',
  'ground',
  'surfac',
  'ultrason',
  'face',
  'grind',
  'silicon',
  'carbid',
  'sic',
  'ceram',
  'minor',
  'vibrat',
  'amplitud'],
 'abstract': []}

<div dir='rtl'>
<h2>1-1.
نمایه‌سازی
</h2>
<p>
در این بخش باید برای سامانه یک
Positional Index
بسازید. 
<br>
با توجه به مواردی که در بخش بعد می‌آید و نیاز به جست‌وجو‌ی مجزا و با امتیازدهی متفاوت بر روی بخش‌های مختلف سند مثل عنوان یا چکیده آن، در این قسمت باید نمایه‌ی مناسب برای امکان جست‌وجو‌ در بخش‌های مختلف را پیاده‌سازی کنید.
با استفاده از نمایه‌ی ساخته‌شده باید بتوان شماره‌ی تمامی اسنادی که یک کلمه در آن آمده است و همچنین همه‌ی جایگاه‌های این کلمه در هر بخش از هر سند را پیدا کرد.

برای این بخش می‌توانید از نمایه‌ای که در تمرین اول زدید استفاده کنید. 
</p>
</div>

In [4]:
class DocumentBatch:
    def __init__(self, corpus: dict[str, dict[str, list[str]]]):
        self.documents = corpus
        self.index = defaultdict(lambda: defaultdict(lambda: {'title': [], 'abstract': []}))

    def index_construction(self):
        # {TODO}: Build an inverted index for the batch
        for document_index, preprocessed_row in self.documents.items():
            title_tokens = preprocessed_row['title']
            abstract_tokens = preprocessed_row['abstract']
            for token_index, token in enumerate(title_tokens):
                self.index[token][document_index]['title'].append(token_index)
            for token_index, token in enumerate(abstract_tokens):
                self.index[token][document_index]['abstract'].append(token_index)


class FastSearchEngine:
    def __init__(self):
        self.main_index = defaultdict(lambda: defaultdict(lambda: {'title': [], 'abstract': []}))
        self.daily_indices = deque()

    def add_batch(self, batch: DocumentBatch):
        # {TODO}: Incorporate the new batch into the daily indices
        batch.index_construction()
        self.daily_indices.append(batch.index)
        
    def end_of_day_logarithmic_merge(self):
        # {TODO}: Implement the logarithmic merge strategy
        while self.daily_indices:
            index = self.daily_indices.pop()
            for key, value in index.items():
                self.main_index[key].update(value)

In [5]:
def construct_positional_indexes(corpus: dict) -> dict:
    """
    Constructs positional indexes for the processed data by inserting words into a trie and creating positional indexes and posting lists.

    Parameters
    ----------
    corpus : dict
        The processed data.

    Returns
    -------
    dict
        The constructed positional indexes.
    """

    postional_index = defaultdict(lambda: defaultdict(lambda: {'title': [], 'abstract': []}))
    # TODO: Insert words from the processed data into a trie and construct positional indexes and posting lists.
    search_engine = FastSearchEngine()
    servers_number = 5
    days_number = 5
    servers_item_list = []
    items_list = list(corpus.items())
    for i in range(servers_number):
        servers_item_list.append(items_list[i::servers_number])
    for i in tqdm(range(days_number)):
        for j in range(servers_number):
            batch = DocumentBatch(defaultdict(lambda: {'title': [], 'abstract': []}, servers_item_list[j][i::days_number]))
            search_engine.add_batch(batch)
        search_engine.end_of_day_logarithmic_merge()

    postional_index.update(search_engine.main_index)
    return postional_index

docs = construct_positional_indexes(corpus)
print(corpus[df.index[0]]['title'][0])
docs[corpus[df.index[0]]['title'][0]]['871057177837ffacc315d31bb9f2dcaa705a0ff0']

  0%|          | 0/5 [00:00<?, ?it/s]

analysi


{'title': [], 'abstract': [46, 154, 155]}

<div dir='rtl'>
<h1>2.
مدل‌های برداری و احتمالاتی
</h1>
<p>
در این بخش قصد داریم تا با استفاده از مدل‌های برداری و احتمالاتی، دو سیستم بازیابی اطلاعات طراحی کنیم. در نهایت قرار است سیستم‌های طراحی شده در این بخش را مورد ارزیابی قرار دهیم.

در مدل‌های برداری ما به ازای هر داک که در اختیار داریم و کوئری ورودی یک بردار در فضا در نظر می‌گیریم. در ادامه برای بدست آوردن میزان ارتباط داک‌ها و کوئری، از معیارهایی مانند ضرب داخلی بردارها استفاده می‌کنیم. در این بخش ابتدا بردارهای مربوط به هرکدام را با استفاده از مقادیر 
tf, idf
می‌سازیم و سپس به سراغ محاسبه امتیاز داک‌ها می‌رویم تا در نهایت بتوانیم با استفاده از این امتیازها داک‌هایی مرتبط‌تر را خروجی دهیم و یک سیستم بازیابی کامل را طراحی کنیم. 

در مدل‌های احتمالاتی اساس کار محاسبه 
$P(R | d, q)$
است. در این روش می‌خواهیم به نوعی داک‌ها را به صورت مدل‌های احتمالاتی ببینیم. در درس مشاهده کردید که در انتها مدل‌های احتمالاتی هم رفتاری مشابه با مدل‌های برداری دارند. در ادامه یک سیستم مبتنی بر این مدل‌ها و با روش 
Okapi25
طراحی می‌کنید. 
</p>
</div>

<div dir='rtl'>
<h2>2-1.
ساخت ماتریس TF-IDF
</h2>
<p>
در این مرحله دو تابع
<code dir="ltr">get_tf(token, doc_id)</code>
و
<code dir="ltr">get_idf(token)</code>
را پیاده‌سازی کنید که تابع اول مقدار
tf
توکن ورودی را در شناسه‌ی موردنظر
و تابع دوم مقدار
idf
توکن ورودی را برروی نمایه‌ی کنونی حساب می‌کند و آن را به عنوان خروجی برمی‌گرداند.
</p>
</div>

$$tf_{t, d} = \text{Numbers of } t \text{ in the title of document } d + \text{Numbers of } t \text{ in the abstract of document } d$$
$$idf_t = \log\Biggl(\frac{\text{Number of documents}}{\text{Number of documents that contains } t + 1}\Biggr)$$

In [6]:
def get_tf(token: str, positional_index: dict, doc_id: str) -> int:
    """
    Retrieves the term frequency (TF) of a given token within a specific document.

    Parameters
    ----------
    token : str
        The token for which to retrieve the term frequency.
    positional_index : dict
        A positional index that maps tokens to their positions in documents.
    doc_id : str
        The unique identifier of the document in which to calculate the TF.

    Returns
    -------
    int
        The term frequency (TF) of the given token within the specified document.
    """

    # TODO: Implement the calculation of term frequency (TF) for the token in the specified document.
    return len(positional_index[token][doc_id]['title']) + len(positional_index[token][doc_id]['abstract'])

get_tf('deep', docs, '40ea606185b59cd07b456cb1022d64bf41f5538d')

0

In [7]:
def get_idf(token: str, positional_index: dict, corpus: dict) -> float:
    """
    Retrieves the inverse document frequency (IDF) of a given token.

    Parameters
    ----------
    token : str
        The token to retrieve the IDF for.

    Returns
    -------
    float
        The IDF of the given token.
    """

    # TODO: Implement the calculation of IDF for the token based on your document collection.
    return np.log(len(corpus) / (len(positional_index[token]) + 1))

get_idf('deep', docs, corpus)

2.4873356041034036

<div dir='rtl'>
<p>
در مرحله‌ی نهایی تابع
<code dir="ltr">generate_tfidf_list(corpus, positional_index)</code>
را پیاده‌سازی کنید که با ورودی گرفتن
corpus
و
positional_index
مقادیر
tf-idf
را برای هر
token
به دست می‌آورد و به شکل یک ماتریس به عنوان خروجی برمی‌گرداند.
</p>
</div>

In [8]:
def generate_tfidf_list(corpus: dict, positional_index: dict) -> list[dict[str, float]]:
    """
    Generates a list of dictionaries representing documents with associated TF-IDF scores.

    Parameters
    ----------
    corpus : dict
        The corpus containing the processed data.
    positional_index : dict
        The positional index containing the term frequencies and document frequencies.

    Returns
    -------
    list[dict[str, float]]
        The list of dictionaries representing documents with associated TF-IDF scores.
    """

    # TODO: Write the function to generate the list of dictionaries with TF-IDF scores.
    tfidf_list = []
    for doc_id, doc in tqdm(corpus.items()):
        tfidf = defaultdict(lambda: 0)
        for token in doc['title'] + doc['abstract']:
            tfidf[token] = get_tf(token, positional_index, doc_id) * get_idf(token, positional_index, corpus)
        tfidf_list.append(tfidf)
    return tfidf_list

tfidf = generate_tfidf_list(corpus, docs)
tfidf[0]

  0%|          | 0/6183 [00:00<?, ?it/s]

defaultdict(<function __main__.generate_tfidf_list.<locals>.<lambda>()>,
            {'analysi': 1.804946473510009,
             'ground': 4.797733236834243,
             'surfac': 3.5820643927451155,
             'ultrason': 6.783648720503256,
             'face': 4.310718261761971,
             'grind': 7.120120957124469,
             'silicon': 5.896345525502353,
             'carbid': 8.036411688998623,
             'sic': 7.630946580890459,
             'ceram': 5.9569701473187875,
             'minor': 5.2031983449424075,
             'vibrat': 5.9569701473187875,
             'amplitud': 5.510683044690368})

In [9]:
def show_document_vector(vector: list[dict[str, float]], doc_index: int) -> None:
    """
    Prints the non-zero weights and corresponding terms for a document represented as a vector.

    Parameters
    ----------
    vector : list of dict of str:float
        The list of document vectors, where each document vector is a dictionary with term weights.
    doc_id : int
        The document index for which to display the vector.

    Returns
    -------
    None
    """
    vector_list = [(term, weight) for term, weight in vector[doc_index].items() if weight > 0]
    vector_list.sort(key=lambda x: x[1], reverse=True)
    length = math.sqrt(sum(weight ** 2 for _, weight in vector_list))
    normalized = {term: tfidf / length for term, tfidf in vector_list}
    for term, tfidf in vector_list:
        print(f'{term.ljust(16)}\t{tfidf}\t(normalized: {normalized[term]})')

show_document_vector(tfidf, 0)

carbid          	8.036411688998623	(normalized: 0.3829479102083639)
sic             	7.630946580890459	(normalized: 0.3636268472985346)
grind           	7.120120957124469	(normalized: 0.33928518678232916)
ultrason        	6.783648720503256	(normalized: 0.32325174488765646)
ceram           	5.9569701473187875	(normalized: 0.283859184592569)
vibrat          	5.9569701473187875	(normalized: 0.283859184592569)
silicon         	5.896345525502353	(normalized: 0.28097032409982475)
amplitud        	5.510683044690368	(normalized: 0.2625928881510294)
minor           	5.2031983449424075	(normalized: 0.24794074889455978)
ground          	4.797733236834243	(normalized: 0.22861968598473048)
face            	4.310718261761971	(normalized: 0.20541264107942597)
surfac          	3.5820643927451155	(normalized: 0.17069111520398728)
analysi         	1.804946473510009	(normalized: 0.08600859523098188)


<div dir='rtl'>
<h2>2-2.
امتیازدهی به سندها
</h2>
<p>
در این بخش می‌خواهیم تا با سه شیوه‌ی متفاوت، داکیومنت‌ها را امتیازدهی کرده و kتا با بالاترین امتیاز را برگردانیم.
روش‌ها به شکل زیر است:

- **ltc.lnc:** در این روش از cosine similarity استفاده می‌کنیم و داده‌ها را بر اساس آن مرتّب می‌کنیم. برای متوجّه شدن نحوه‌ی عملکرد این سیستم، می‌توانید به اسلایدهای درس مراجعه کنید. همچنین دقت کنید که لازم است این امتیازدهی را هم به روش 
term-at-a-time
و هم به روش 
doc-at-a-time
محاسبه کنید.

- **Okapi25:** برای پیاده‌سازی این روش از تساوی زیر استفاده کنید:
</p>
</div>
<div>
<p>

$$ RSV_d = \sum_{t \in q} idf(t)\times\frac{(k_1 + 1)tf(t, d)}{k_1((1 - b) + b\frac{dl(d)}{avg(dl)}) + tf(t, d)} $$
</p>
</div>
<div dir='rtl'>
<p>
برای محاسبه این دو معیار، از توابعی که در بخش قبل پیاده‌سازی کردید استفاده کنید.

در تابع search که مربوط به جست و جوی پرسمان کلی است، به عنوان ورودی پارامتر پرسمان (query)، روش محاسبه امتیاز (method)، تعداد اسنادی که باید برگردانده شود (max-result-count) را ورودی می گیرید. ورودی وزن 
(weight)
نشان می‌دهد که امتیاز نهایی چه وزنی از امتیاز عنوان و چکیده دارد. به زبان دیگر:
</p>
</div>
<div>
<p>

$$ final\_score = weight \times title\_score + (1 - weight) \times abstract\_score $$
</p>
</div>
<div dir='rtl'>
<p>

**توجّه کنید** که تابع نوشته شده، صرفاً یک prototype است و تا وقتی که نیازمندی‌های گفته شده را پیاده کنید، تغییر تابع مشکلی ندارد.
</p>
</div>

<div dir='rtl'>

# پاسخ:
### با توجه به اینکه تابع های قبلی با نیاز های مسئله مطابقت ندارد در ادامه برای امتیاز دهی به سند‌‌ها، اینجانب به ازای هر روش، یک dataframe جداگانه محاسبه میکنم و در هنگام مشخص شدن مقدار پرسش به کمک آنها امتیاز سند را مشخص می‌نمایم 
</div>

In [10]:
def get_tokens_tf_idf_matrix():
    cols = pd.MultiIndex.from_product([list(corpus.keys()) + ['idf'], ['title', 'abstract']])
    data = np.zeros((len(docs.keys()), len(cols)))
    tokens_tf_idf_matrix = pd.DataFrame(data, index=docs.keys(), columns=cols)
    tokens_tf_idf_matrix.columns = cols

    indexes_place = {index: ind_of_index for ind_of_index, index in enumerate(tokens_tf_idf_matrix.index)}
    columns_place = {}
    for index, (doc_id, doc_part) in enumerate(tokens_tf_idf_matrix.columns):
        if doc_id not in columns_place:
            columns_place[doc_id] = {}
        columns_place[doc_id][doc_part] = index
        
    tf_idf_matrix_np = tokens_tf_idf_matrix.to_numpy()

    for token, doc_dict in tqdm(docs.items()):
        token_df = {'title': 0, 'abstract': 0}
        for doc_id, incidence_dict in doc_dict.items():
            for document_part, incidence_list in incidence_dict.items():
                index = indexes_place[token]
                column = columns_place[doc_id][document_part]
                tf_idf_matrix_np[index, column] = len(incidence_list)
                if incidence_list:
                    token_df[document_part] += 1
        for document_part, document_part_token_df in token_df.items():
            index = indexes_place[token]
            column = columns_place['idf'][document_part]
            tf_idf_matrix_np[index, column] = np.log(len(corpus) / (document_part_token_df + 1))
            
    return tokens_tf_idf_matrix

tokens_tf_idf_matrix = get_tokens_tf_idf_matrix()
tokens_tf_idf_matrix.head(1)

  0%|          | 0/20610 [00:00<?, ?it/s]

40ea606185b59cd07b456cb1022d64bf41f5538d           \
                                            title abstract   
interact                                      0.0      0.0   

         597c2d96c45e8ad83fc08e5d464d266b68f873ed           \
                                            title abstract   
interact                                      0.0      0.0   

         afbf330f0180320deff12fe42ded4f087b4a1811           \
                                            title abstract   
interact                                      0.0      0.0   

         fa57e1cbaa46211499749cc75172e9ced26ad539           \
                                            title abstract   
interact                                      0.0      0.0   

         8780ce7c0d09d36d293def6be23e64b12c644169           ...  \
                                            title abstract  ...   
interact                                      1.0      0.0  ...   

         41156a34679d10200ad4bc5ba3fb391c484417ab           \
                                            title abstract   
interact                                      0.0      0.0   

         7cc4ecd09745b8aec00bfe07618db4d6be05cca2           \
                                            title abstract   
interact                                      0.0      0.0   

         188a93c03c4db17c487ccc9419864d341acf0bb9           \
                                            title abstract   
interact                                      0.0      0.0   

         812bf641028468bcfe95d31d515129728e067d99                idf            
                                            title abstract     title  abstract  
interact                                      0.0      0.0  4.085168  2.778916  

[1 rows x 12368 columns]

In [11]:
def df_ind(*args):
    return pd.MultiIndex.from_product(args)

In [12]:
def get_docs_tf_idf_matrix():
    docs_tf_idf_matrix = tokens_tf_idf_matrix[corpus.keys()].copy(deep=True)

    # TF calculation
    docs_tf_idf_matrix[:] = 1 + np.where(docs_tf_idf_matrix.to_numpy() > 0, np.log(docs_tf_idf_matrix.to_numpy()), 0)

    # IDF multiplication
    np_docs_title_tf_idf_matrix = docs_tf_idf_matrix[df_ind(corpus.keys(), ['title'])].to_numpy()
    np_docs_abstract_tf_idf_matrix = docs_tf_idf_matrix[df_ind(corpus.keys(), ['abstract'])].to_numpy()

    np_docs_title_tf_idf_matrix = np_docs_title_tf_idf_matrix * tokens_tf_idf_matrix[('idf', 'title')].to_numpy()[:, np.newaxis]
    np_docs_abstract_tf_idf_matrix = np_docs_abstract_tf_idf_matrix * tokens_tf_idf_matrix[('idf', 'abstract')].to_numpy()[:, np.newaxis]

    docs_tf_idf_matrix[df_ind(corpus.keys(), ['title'])] = np_docs_title_tf_idf_matrix
    docs_tf_idf_matrix[df_ind(corpus.keys(), ['abstract'])] = np_docs_abstract_tf_idf_matrix

    # norm caculation
    np_docs_tf_idf_matrix = docs_tf_idf_matrix.to_numpy()

    docs_tf_idf_matrix[:] = np.divide(
        np_docs_tf_idf_matrix,
        (np_docs_tf_idf_matrix**2).sum(axis=0)[np.newaxis, :]**(1/2)
    )
    
    return docs_tf_idf_matrix

docs_tf_idf_matrix = get_docs_tf_idf_matrix()
# show final result
(docs_tf_idf_matrix.to_numpy()**2).sum(axis=0)[:10]

/tmp/ipykernel_18783/3293177143.py:5: RuntimeWarning: divide by zero encountered in log
  docs_tf_idf_matrix[:] = 1 + np.where(docs_tf_idf_matrix.to_numpy() > 0, np.log(docs_tf_idf_matrix.to_numpy()), 0)


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [13]:
b = 0.75
k1 = np.mean([1.2, 2])

@lru_cache(maxsize=None)
def get_avg_doc_len_and_len_corpus():
    len_corpus = {}
    mean_doc_len = {'title': 0, 'abstract': 0}
    for doc_id, doc_corpus in corpus.items():
        len_corpus[doc_id] = {}
        for doc_part_label, doc_part_list in doc_corpus.items():
            len_corpus[doc_id][doc_part_label] = len(doc_part_list)
            mean_doc_len[doc_part_label] += len(doc_part_list)
    mean_doc_len['title'] /= len(corpus)
    mean_doc_len['abstract'] /= len(corpus)
    
    return mean_doc_len, len_corpus

def get_np_docs_rsv_matrix():
    mean_doc_len, len_corpus = get_avg_doc_len_and_len_corpus()

    dividend_coefficient = k1 + 1
    doc_divisor_reminder = {'title': [], 'abstract': []}
    for doc_id in corpus.keys():
        doc_len_corpus = len_corpus[doc_id]
        for doc_part_label, doc_part_len in doc_len_corpus.items():
            doc_divisor_reminder[doc_part_label].append(k1 * (1-b + b * doc_part_len / mean_doc_len[doc_part_label]))

    docs_rsv_matrix = tokens_tf_idf_matrix[corpus.keys()].copy(deep=True)

    np_docs_rsv_matrix = docs_rsv_matrix.to_numpy()
    np_docs_rsv_matrix[:, 0::2] = np.divide(
        np_docs_rsv_matrix[:, 0::2] * dividend_coefficient,
        (
            np_docs_rsv_matrix[:, 0::2] 
            + np.array(doc_divisor_reminder['title'])[np.newaxis, :]
        )
    )* tokens_tf_idf_matrix['idf']['title'].to_numpy()[:, np.newaxis]
    np_docs_rsv_matrix[:, 1::2] = np.divide(
        np_docs_rsv_matrix[:, 1::2] * dividend_coefficient, 
        (
            np_docs_rsv_matrix[:, 1::2] 
            + np.array(doc_divisor_reminder['abstract'])[np.newaxis, :]
        )
    ) * tokens_tf_idf_matrix['idf']['abstract'].to_numpy()[:, np.newaxis]
    
    return docs_rsv_matrix

docs_rsv_matrix = get_np_docs_rsv_matrix()

In [14]:
def get_query_lnc(query: str):
    q_tokens = preprocessor.preprocess(query)
    q_tokens_dict = defaultdict(lambda: 0)
    q_tokens_dict.update(Counter(q_tokens))
    q_vector_square_sum = 0
    for token, count in q_tokens_dict.items():
        q_tokens_dict[token] = (1 + np.log(count))
        q_vector_square_sum += q_tokens_dict[token]**2
    q_vector_length = q_vector_square_sum**(1/2)
    for token, tf in q_tokens_dict.items():
        q_tokens_dict[token] = tf / q_vector_length
    
    return q_tokens_dict

def search(query: str, max_result_count: int, method: str = 'ltc-lnc-tat', weight: float = 0.5):
    """
    Finds relevant documents to query

    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                        notice that if max_result_count = -1, then you have to return all docs

    method: 'ltc-lnc-tat' or 'ltc-lnc-dat' or 'okapi25'

    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retreived documents with snippet
    """
    scores = defaultdict(lambda: 0)
    if method == 'ltc-lnc-tat':
        q_tokens_dict = get_query_lnc(query)
        for token, lnc in q_tokens_dict.items():
            for doc_id in corpus.keys():
                scores[doc_id] += (
                    weight * lnc * docs_tf_idf_matrix[doc_id, 'title'][token] 
                    + (1 - weight) * lnc * docs_tf_idf_matrix[doc_id, 'abstract'][token]
                )
    elif method == 'ltc-lnc-dat':
        q_tokens_dict = get_query_lnc(query)
        for doc_id in corpus.keys():
            for token, lnc in q_tokens_dict.items():
                scores[doc_id] += (
                    weight * lnc * docs_tf_idf_matrix[doc_id, 'title'][token] 
                    + (1 - weight) * lnc * docs_tf_idf_matrix[doc_id, 'abstract'][token]
                )
    else:
        q_tokens = preprocessor.preprocess(query)
        for doc_id in corpus.keys():
            for token in q_tokens:
                scores[doc_id] += (
                    weight * docs_rsv_matrix[doc_id, 'title'][token] 
                    + (1 - weight) * docs_rsv_matrix[doc_id, 'abstract'][token]
                )
    top_k = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:max_result_count]
    return [doc_id for doc_id, _ in top_k]
    

<div dir='rtl'>
<h1>3.
مدل‌های زبانی
</h1>
<p>
در این بخش یک سیستم مبتنی بر مدل‌های زبانی پیاده‌سازی می‌کنید. مدل زبانی‌ای که باید استفاده کنید مدل 
unigram
است که می‌توان گفت ساده‌ترین مدل ممکن است. در این مدل ما برای هر داک یک مدل دظر نظر می‌گیریم و احتمال حضور کلمات در مدل یک داک 
نسبت به هم مستقل هستند. در پیاده‌سازی این بخش دستتان خیلی باز است. مدل‌هایی که تشکیل می‌دهید و نحوه 
smoothing 
و همچنین پارامترهای مدل همگی به خودتان برمی‌گردد. 
</p>
</div>

In [15]:
def get_tokens_dirichlet_smoothing_matrix():
    alpha = 0.5

    tokens_dirichlet_smoothing_matrix = tokens_tf_idf_matrix[corpus.keys()].copy(deep=True)
    
    np_tokens_dirichlet_smoothing_matrix = tokens_dirichlet_smoothing_matrix.to_numpy()
    np_title_collection_freq_matrix = np_tokens_dirichlet_smoothing_matrix[:, 0::2].sum(axis=1)
    np_abstract_collection_freq_matrix = np_tokens_dirichlet_smoothing_matrix[:, 1::2].sum(axis=1)
    np_title_dirichlet_smoothing_matrix = np_title_collection_freq_matrix / np_title_collection_freq_matrix.sum()
    np_abstract_dirichlet_smoothing_matrix = np_abstract_collection_freq_matrix / np_abstract_collection_freq_matrix.sum()


    _, len_corpus = get_avg_doc_len_and_len_corpus()

    len_corpus_np = {'title': [], 'abstract': []}
    for doc_id in corpus.keys():
        doc_len_corpus = len_corpus[doc_id]
        for doc_part_label, doc_part_len in doc_len_corpus.items():
            len_corpus_np[doc_part_label].append(doc_part_len)
    len_corpus_np['title'] = np.array(len_corpus_np['title'])
    len_corpus_np['abstract'] = np.array(len_corpus_np['abstract'])


    np_tokens_dirichlet_smoothing_matrix[:, 0::2] += alpha * np_title_dirichlet_smoothing_matrix[:, np.newaxis] 
    np_tokens_dirichlet_smoothing_matrix[:, 1::2] += alpha * np_abstract_dirichlet_smoothing_matrix[:, np.newaxis] 

    np_tokens_dirichlet_smoothing_matrix[:, 0::2] = np.divide(
        np_tokens_dirichlet_smoothing_matrix[:, 0::2],
        len_corpus_np['title'][np.newaxis, :] + alpha
    )
    np_tokens_dirichlet_smoothing_matrix[:, 1::2] = np.divide(
        np_tokens_dirichlet_smoothing_matrix[:, 1::2],
        len_corpus_np['abstract'][np.newaxis, :] + alpha
    )
    
    return tokens_dirichlet_smoothing_matrix

tokens_dirichlet_smoothing_matrix = get_tokens_dirichlet_smoothing_matrix()

In [16]:
def lm_search(query: str, max_result_count: int):
    """
    Finds relevant documents to query

    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                        notice that if max_result_count = -1, then you have to return all docs

    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retreived documents with snippet
    """
    weight = 0.5
    scores = defaultdict(lambda: 1)
    q_tokens = preprocessor.preprocess(query)
    for doc_id in corpus.keys():
        title_score = weight
        abstract_score = (1-weight)
        for token in q_tokens:
            title_score *= tokens_dirichlet_smoothing_matrix[doc_id, 'title'][token]
            abstract_score *= tokens_dirichlet_smoothing_matrix[doc_id, 'abstract'][token]
        scores[doc_id] = title_score + abstract_score
    top_k = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:max_result_count]
    return [doc_id for doc_id, _ in top_k]

<div dir='rtl'>
<h1>4.
ارزیابی عملکرد سامانه
</h1>
<p>
در این بخش معیارهای زیر را برای سیستم‌های طراحی شده پیاده‌سازی کنید. سپس در بخش آخر با فراخوانی توابع پیاده‌سازی شده و ورودی دادن مقادیر موجود در فایل
validation.json
به عنوان ورودی 
actual
توابع، می‌توانید معیارها را برای سیستم‌های بازیابی خودتان بدست بیاورید. 
</p>
</div>

In [17]:
class Evaluation:
    def __init__(self, actual, predicts):
        self.actual = actual
        self.queries = predicts

    def evaluate(self, require_scores):
        """
        Prints require scores for actual and predicts array

        Parameters
        ----------
        require_scores : List[str]
            Scores required to calculated

        Returns
        -------
        """
        ["precision", "recall", "f1", "map", "ndcg", "mrr"]
        for require_score in require_scores:
            scores = None
            if require_score == 'precision':
                scores = self.precision()
            elif require_score == 'recall':
                scores = self.recall()
            elif require_score == 'f1':
                scores = self.f1()
            elif require_score == 'map':
                scores = self.map()
            elif require_score == 'ndcg':
                scores = self.ndcg()
            elif require_score == 'mrr':
                scores = self.mrr()
            else:
                continue
            print(f'{require_score}: {np.mean(scores)}')

    def precision(self):
        """
        Calculates the precision of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The precision of the predicted results
        """
        precisions = []
        for i in range(len(self.queries)):
            query_top_k = self.queries[i]
            actual_top_k = self.actual[i]
            precisions.append(
                len(set(query_top_k) & set(actual_top_k)) 
                /
                len(query_top_k)
            )
        
        return precisions

    def recall(self):
        """
        Calculates the recall of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The recall of the predicted results
        """
        recalls = []
        for i in range(len(self.queries)):
            query_top_k = self.queries[i]
            actual_top_k = self.actual[i]
            recalls.append(
                len(set(query_top_k) & set(actual_top_k)) 
                /
                len(actual_top_k)
            )

        return recalls

    def f1(self):
        """
        Calculates the F1 score of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The F1 score of the predicted results
        """
        f1_scores = []
        precision = self.precision()
        recall = self.recall()
        for i in range(len(precision)):
            f1_scores.append(
                2 * precision[i] * recall[i] 
                /
                (precision[i] + recall[i])
            )

        return f1_scores

    def map(self):
        """
        Calculates the Mean Average Precision of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The Mean Average Precision of the predicted results
        """
        map_scores = []
        for j in range(len(self.queries)):
            query_top_k = self.queries[j]
            actual_top_k = self.actual[j]
            correct_selections_set = set(query_top_k) & set(actual_top_k)
            m_j = len(correct_selections_set)
            map_scores.append(0)
            tp_conter = 0
            for i in range(len(query_top_k)):
                if query_top_k[i] in correct_selections_set:
                    tp_conter += 1
                    map_scores[j] += tp_conter / (i + 1) / m_j
            

        return map_scores

    def ndcg(self):
        """
        Calculates the Normalized Discounted Cumulative Gain (NDCG) of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The NDCG of the predicted results
        """
        ndcg_scores = []
        for j in range(len(self.queries)):
            query_top_k = self.queries[j]
            actual_top_k_set = set(self.actual[j])
            normalization_factor = 0
            ndcg_scores.append(0)
            for m in range(len(query_top_k)):
                value = 1 / np.log2(2 + m)
                normalization_factor += value
                if query_top_k[m] in actual_top_k_set:
                    ndcg_scores[j] += value
            ndcg_scores[j] /= normalization_factor

        return ndcg_scores

    def mrr(self):
        """
        Calculates the Mean Reciprocal Rank of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The MRR of the predicted results
        """
        mrr_scores = []
        for i in range(len(self.queries)):
            query_top_k = self.queries[i]
            actual_top_k = self.actual[i]
            actual_top_1 = actual_top_k[0]
            try:
                index = query_top_k.index(actual_top_1)
                mrr_scores.append(1 / (index + 1))
            except ValueError:
                mrr_scores.append(0)                

        return mrr_scores

In [18]:
require_scores = ["precision", "recall", "f1", "map", "ndcg", "mrr"]

# Read actuals and queries from validation.json file
actuals = []
queries = []

with open('validation.json', 'r') as file:
    val_json = json.loads(file.read())
actuals.extend(val_json.values())
queries.extend(val_json.keys())


In [19]:
predicts = [search(query, 10, method='ltc-lnc-tat') for query in queries]

eval_vsm = Evaluation(actuals, predicts)
eval_vsm.evaluate(require_scores)

precision: 0.3666666666666667
recall: 0.3666666666666667
f1: 0.3666666666666667
map: 0.5331569664902999
ndcg: 0.3947057678195898
mrr: 0.2740740740740741


In [20]:
predicts = [search(query, 10, method='okapi25') for query in queries]

eval_bm = Evaluation(actuals, predicts)
eval_bm.evaluate(require_scores)

precision: 0.65
recall: 0.65
f1: 0.65
map: 0.8044690098261528
ndcg: 0.6878623965624748
mrr: 0.32010582010582006


In [21]:
predicts = [lm_search(query, 10) for query in queries]

eval_lm = Evaluation(actuals, predicts)
eval_lm.evaluate(require_scores)

precision: 0.75
recall: 0.75
f1: 0.75
map: 0.8724546222810111
ndcg: 0.7696766746699987
mrr: 0.2876984126984127
